In [6]:
import sqlite3
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from IPython.core.debugger import set_trace

np.random.seed(42)
max_rows_to_expand = 10000

In [48]:
user_grouped_rating_files = [csv_file for csv_file in glob('datasets/user_grouped_ratings/user_grouped_ratings_processed*.pkl')]
user_grouped_rating_files.sort()

In [49]:
def extract_required_format(record):
    new_rows = []
    history = 5
    prediction = 1
    seq_length = history + prediction
    record_df = pd.DataFrame({
        'anime_id': record['anime_id'],
        'my_score': record['my_score'],
        'user_id': record['user_id']
    })
    size = min([len(record_df) // seq_length, max_rows_to_expand])
    for _ in range(size):
        indexes = np.random.choice(record_df.index, size=seq_length)
        train = record_df.iloc[indexes]
        new_rows.append({
            'anime_id': train['anime_id'].values,
            'user_id': train['user_id'].values,
            'my_score': train['my_score'].values
        })
    return new_rows

In [11]:
for index, rating_file in tqdm(enumerate(user_grouped_rating_files)):
    d = pd.read_pickle(rating_file).reset_index()
    df = d[d['anime_id'].map(lambda x: len(x) > 5)]
    new_rows = []
    for idx, record in df.iterrows():
        new_rows.extend(extract_required_format(record))
    new_df = pd.DataFrame(new_rows)
    new_df.to_pickle(
        'datasets/user_grouped_ratings/augmented_{}_user_grouped_ratings_processed_{}.pkl'.format(max_rows_to_expand, index),
        compression=None
    )
    del(new_df)
    del(df)
    del(d)
    del(new_rows)

11it [34:15, 149.85s/it]


In [50]:
import pickle, sqlite3

def chunks(data, rows=10000):
    """ Divides the data into 10000 rows each """

#     for i in range(0, len(data), rows):
#         yield data[i:i+rows]

# def write_to_sqlite(sqlite, data):
#     conn = sqlite3.connect(sqlite)
#     conn.text_factory = str  #bugger 8-bit bytestrings
#     cur = conn.cursor()
#     cur.execute('CREATE TABLE IF NOT EXISTS data (blob VARCHAR)')
    

#     divData = chunks(data) # divide into 10000 rows each

#     for chunk in divData:
#         set_trace()
#         break
#         cur.execute('BEGIN TRANSACTION')

#         for field in chunk:
#             cur.execute('INSERT OR IGNORE INTO data (blob) VALUES (?)', (pickle.dumps(field)))

#         cur.execute('COMMIT')

for pkl_file in tqdm(glob('datasets/user_grouped_ratings/augmented_{}_user_grouped_ratings_processed_*.pkl'.format(max_rows_to_expand))):
    sqlite = pkl_file.replace('.pkl', '.db')
    with open(pkl_file, 'rb') as fp:
        data = pickle.load(fp)
    data['blob'] = data.apply(lambda x: pickle.dumps(x), axis=1)
    data[['blob']].to_sql(name='augmented_data', if_exists='replace', con=sqlite3.connect(sqlite), chunksize=10000)
#     write_to_sqlite(sqlite, data[:10])





  0%|          | 0/11 [00:00<?, ?it/s]



  9%|▉         | 1/11 [01:00<10:08, 60.87s/it]



 18%|█▊        | 2/11 [01:58<09:00, 60.05s/it]



 27%|██▋       | 3/11 [02:29<06:49, 51.16s/it]



 36%|███▋      | 4/11 [03:07<05:30, 47.28s/it]



 45%|████▌     | 5/11 [03:52<04:39, 46.58s/it]



 55%|█████▍    | 6/11 [04:23<03:28, 41.73s/it]



 64%|██████▎   | 7/11 [05:35<03:23, 50.82s/it]

KeyboardInterrupt: 

In [2]:
import pickle, sqlite3

for pkl_file in tqdm(glob('datasets/user_grouped_ratings/user_grouped_ratings_processed_*.pkl')):
    sqlite = pkl_file.replace('.pkl', '.db')
    with open(pkl_file, 'rb') as fp:
        data = pickle.load(fp)
    data['blob'] = data.apply(lambda x: pickle.dumps(x), axis=1)
    data[['blob']].to_sql(name='user_grouped', if_exists='replace', con=sqlite3.connect(sqlite), chunksize=10000)
#     write_to_sqlite(sqlite, data[:10])

100%|██████████| 11/11 [00:20<00:00,  1.79s/it]


In [10]:
import pandas as pd
import sqlite3

for pkl_file in tqdm(glob('datasets/processed_ratings/user_ratings_processed_*.csv')):
    sqlite = pkl_file.replace('.csv', '.db')
    pd.read_csv(pkl_file).to_sql(
        name='user_ratings',
        if_exists='replace',
        con=sqlite3.connect(sqlite),
        chunksize=10000)
#     write_to_sqlite(sqlite, data[:10])

100%|██████████| 32/32 [01:31<00:00,  2.96s/it]
